In [ ]:
require 'httpclient'
require 'rest-client'
require 'json'
require 'pp'
require 'awesome_print'

config = {
  scheme: 'http',
  hostname: 'localhost',
  #hostname: 'docker.for.mac.localhost', # if you are on Docker for Mac
  port: 8080
  }

admin = {
  username: 'admin@everest.engineering',
  password: 'ac0n3x72'
  }

org1_user1 = {
  username: 'org1user1@example.com',
  password: 'justauser'
  }

self_registered_user1 = {
  username: 'selfregistered1@example.com',
  password: 'justauser'
  }


# System

## Version info


In [ ]:
response = RestClient.get "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/version"
response.body

## Admin login


In [ ]:
response = RestClient.post "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/oauth/token?grant_type=password&client_id=web-app-ui&username=#{admin[:username]}&password=#{admin[:password]}", {}
admin_token = JSON.parse(response.body)["access_token"]

## System metric endpoints

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/actuator/metrics",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
ap JSON.parse(response.body)["names"].sort

## System metrics - HTTP server requests

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/actuator/metrics/http.server.requests",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
ap JSON.parse(response.body)

## System metrics - Prometheus

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/actuator/prometheus",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
print(response.body)

## Replay status

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/actuator/replay",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
ap JSON.parse(response.body)

## Trigger replay

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/actuator/replay",
{

}.to_json,
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
ap response.body

# Admin

## Create organisation


In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/admin/organizations",
{
  organizationName: "ACME Corporation",
  contactEmail: "org-admin2@example.com"
}.to_json,
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgId = JSON.parse(response.body)

## List organisations

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/admin/organizations",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgList = JSON.parse(response.body)
ap orgList

## Create organisation user

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/#{orgId}/users",
{
  username: "#{org1_user1[:username]}",
  displayName: "Loyal employee",
  password: "#{org1_user1[:password]}"
}.to_json,
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
org1_user1_jwt = JSON.parse(response.body)

## List organisation users

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/#{orgId}/users",
{
  content_type: :json,
  Authorization: "Bearer #{admin_token}"
})
orgUsers = JSON.parse(response.body)
ap orgUsers

# User


## Login as organisation user

In [ ]:
response = RestClient.post "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/oauth/token?grant_type=password&client_id=web-app-ui&username=#{org1_user1[:username]}&password=#{org1_user1[:password]}", {}
org1_user1_jwt = JSON.parse(response.body)["access_token"]

## Upload profile photo

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/user/profile-photo",
{
  file: File.new("./images/profile.jpg", 'rb'),
},
{
  
  Authorization: "Bearer #{org1_user1_jwt}"
})

## Download (stream) profile photo thumbnail

In [ ]:
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/user/profile-photo/thumbnail?width=127&height=128",
{
  content_type: :json,
  Authorization: "Bearer #{org1_user1_jwt}"
})
File.open("/tmp/downloaded-profile-thumbnail.jpg", "w") {|f| f.write(response.to_s) }
File.open("/tmp/downloaded-profile-thumbnail.jpg")

# Self Registered Organisation Admin

## Register a new organisation

In [ ]:
response = RestClient.post("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/register",
{
  contactEmail: "#{self_registered_user1[:username]}",
  contactName: "Organisation Admin",
  contactPassword: "#{self_registered_user1[:password]}",
  organizationName: "Super ACME Company"
}.to_json,
{
  content_type: :json,
})
registeredOrgId = JSON.parse(response.body)["newOrganizationId"]
registeredUserId = JSON.parse(response.body)["newUserId"]

## Confirm registration

In [ ]:
confirmationCode = "paste your confirmation code here"
response = RestClient.get("#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/api/organizations/#{registeredOrgId}/register/#{confirmationCode}",
{
  content_type: :json,
})

## Log in as admin of organisation

In [ ]:
response = RestClient.post "#{config[:scheme]}://#{config[:hostname]}:#{config[:port]}/oauth/token?grant_type=password&client_id=web-app-ui&username=#{self_registered_user1[:username]}&password=#{self_registered_user1[:password]}", {}
self_registered_user1_jwt = JSON.parse(response.body)["access_token"]